In [1]:
import random
import numpy as np
#import pandas as pd

#import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F

import time

/home/miguel/trabajo_grado/Proyecto_GNN/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
def accuracy(pred_y, y):
    
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data, mask):
    
    model.eval()
    out = model(data.x, data.edge_index)
    acc = accuracy(torch.argmax(out, dim = 1)[mask], data.y[mask])
    return acc

def train(model, data, epoch, enable):
    
    f_loss = nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = epoch

    model.train()
    for epoch in range(epochs+1):
        
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = f_loss(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(torch.argmax(out, dim = 1)[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        
        
        val_acc = test(model, data, data.val_mask)

        if enable == True:
          if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% |' f'Val Acc: {val_acc*100:.2f}%')
          
    return model

In [4]:
print(torch.__version__)

1.13.1+cu117


In [5]:
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv, SAGEConv

In [6]:
class GAT(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out, heads=10):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.001,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.5, training=self.training)
    h = self.gat1(h, edge_index)
    h = F.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.gat2(h, edge_index)
  
    return h

In [7]:
class GCN(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.gcn1 = GCNConv(dim_in, dim_h)
    self.gcn2 = GCNConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.001,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.5, training=self.training)
    h = self.gcn1(h, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.gcn2(h, edge_index)
    
    return h

In [8]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.001,
                                      weight_decay=5e-4)
    
    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.5)
        h = self.conv1(h, edge_index)
        h = F.relu(h)
        h = F.dropout(h, p=0.5)
        h = self.conv2(h, edge_index)
       
        return h

In [9]:
from torch_geometric.data import Dataset, Data
from torch_geometric.nn import knn_graph

import os.path as osp
#import pandas as pd
import torch




class MyOwnDatasetFixedKnn(Dataset):
    def __init__(self, root, path, enable = False , transform=None, pre_transform=None, pre_filter=None):
      self.path = path
      self.enable = enable
      super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return self.path

    @property
    def processed_file_names(self):
      if self.enable == False:
        return [f'data_{i}.pt' for i in range(10)]
      else: 
        return [f'data_{i}.pt' for i in range(109)]

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [14]:
from torch_geometric.loader import DataLoader

path1 = "UrbanSound8K_8276.csv"
path2 = "TF_PANNs_8276.hdf5"
dataset = MyOwnDatasetFixedKnn(root = "./own_dataset/data_PANNs_fixed_knn", path = [path1,path2])
loader_dataset = DataLoader(dataset) 

In [15]:
dataset

MyOwnDatasetFixedKnn(10)

In [16]:
#GAT

  
prom_acc_gat = []
dev_values_gat = []
num_epochs_gat = []

#GCN


prom_acc_gcn = []
dev_values_gcn = []
num_epochs_gcn = []

#SAGE


prom_acc_sage = []
dev_values_sage = []
num_epochs_sage = []

In [ ]:
for i in range(30,2230,200):
#GAT

  acc_gat = []
 

#GCN

  acc_gcn = []


#SAGE

  acc_sage = []


  for n, graphs in enumerate(loader_dataset):
#==========================GAT===============================================
      graphs.to(device)
      gat = GAT(2048, 20, 10).to(device)
      train(gat, graphs, epoch = i, enable = False)
      acc_gat.append(test(gat, graphs, graphs.test_mask))

#==========================GCN===============================================  

      
      gcn = GCN(2048, 20, 10).to(device)
      train(gcn, graphs, epoch = i, enable = False)
      acc_gcn.append(test(gcn, graphs, graphs.test_mask))

#==========================SAGE=============================================== 

      
      g_sage = GraphSAGE(2048, 20, 10).to(device)
      train(g_sage, graphs, epoch = i, enable = False)
      acc_sage.append(test(g_sage, graphs, graphs.test_mask))

#==========================GAT===============================================

  prom_acc_gat.append(np.mean(acc_gat))
  dev_values_gat.append(np.std(acc_gat))
  num_epochs_gat.append(i)

#==========================GCN===============================================  

  prom_acc_gcn.append(np.mean(acc_gcn))
  dev_values_gcn.append(np.std(acc_gcn))
  num_epochs_gcn.append(i)

#==========================SAGE===============================================  

  prom_acc_sage.append(np.mean(acc_sage))
  dev_values_sage.append(np.std(acc_sage))
  num_epochs_sage.append(i)

  print(f"GAT\n accuracy mean: {np.mean(acc_gat)*100:.2f}% | standard deviation: {np.std(acc_gat):.2f} | epochs: {i}\n"
        f"GCN\n accuracy mean: {np.mean(acc_gcn)*100:.2f}% | standard deviation: {np.std(acc_gcn):.2f} | epochs: {i}\n"
        f"SAGE\n accuracy mean: {np.mean(acc_sage)*100:.2f}% | standard deviation: {np.std(acc_sage):.2f} | epochs: {i}\n\n"
        f"==============================================================================================================")